<a href="https://colab.research.google.com/github/bob8dod/NLP_project-Review_Analysis/blob/main/Crawling/Crawling_for_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 크롤링(11번가)

In [1]:
import json
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import time
from multiprocessing import Pool, Manager


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
products = 2211689236

url_basic = 'https://www.11st.co.kr/products/{}'.format(products)
data = requests.get(url_basic, headers=headers)
soup = BeautifulSoup(data.text, 'html.parser')

category_path = soup.find('div', attrs={'class':'c_product_category_path'}).find_all('em', attrs={'class':'selected'})
categories = set()
for cate in category_path:
    categories.add(cate.text)
product_name = soup.find('h1', attrs={'class':'title'}).text.strip()
img_src = soup.find('div', attrs={'class':'img_full'}).find('img')['src']
price = soup.find('ul', attrs={'class':'price_wrap'}).find('span',attrs={'class':'value'}).text

manager = Manager()
text = manager.list()
score = manager.list()

def Crawling(pageNo):
    try:
        url = 'https://m.11st.co.kr/products/v1/app/products/{}/reviews/list?pageNo={}&sortType=01&pntVals=&rtype=&themeNm='.format(products, pageNo)
        response = urlopen(url)
        json_data = json.load(response)
        for i in range(len(json_data['review']['list'])):
            if json_data['review']['list'][i]['subject']:
                text.append(''.join([str(pageNo),'|', json_data['review']['list'][i]['subject']]))
                score.append(json_data['review']['list'][i]['evlPnt'])
    except:
        print('리뷰가 끝났습니다. 총 개수:', num)

if __name__ == '__main__':
    start_time = time.time()
    pool = Pool(processes=32)
    pool.map(Crawling,range(20))
    pool.close()
    pool.join()
    print('상품명:', product_name)
    print('카테고리:', categories)
    print('가격:', price)
    print('수집한 리뷰 총 개수: ', len(text))
    print("실행 시간 : %s초" % (time.time() - start_time))

상품명: 프레시지 소고기 샤브샤브 (2인분) 밀키트 쿠킹박스
카테고리: {'식품'}
가격: 12,520
수집한 리뷰 총 개수:  190
실행 시간 : 1.927903413772583초


## New Text2Csv


In [4]:
import pandas as pd
line = [[r.split('|')[0],r.split('|')[1]] for r in text if len(r.split('|'))>1]
data = pd.DataFrame(line, columns=['pageNo','review'])
print('리뷰데이터 총 길이: ', len(data))
data.head(3)

리뷰데이터 총 길이:  190


,pageNo,review
0,10,언제 먹어도 질리지 않네요. 야채 추가로 넘 배부르게 먹었어요. 개인적으로 칼국수도...
1,10,3번째 구매예요~~우리아들이 샤브샤브를좋아해서 가끔 노래를 부르는데 밖에나가서 먹는...
2,10,배송은 예정일보다 하루 늦었지만 야채랑 고기랑 신선하고 좋아서 맛있게 잘 먹었어요....


In [10]:
#csv로 저장 -> {이름} 리뷰({리뷰개수}개)
data.to_csv('{} 리뷰({}개).csv'.format(product_name,len(data)), index=False )